<!-- Pre-processing
1) Create all the pickel files
2) Transfer all that files into filtered folder
3) Convert png to tif
4) create a spatial folder 
5) Put jason and tissue position file in spatial folder
6) put tif files in the input folder
7) do the pre-processin 
8) give counts and meta data to tejas-->

In [165]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import h5py
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from scipy.sparse import csr_matrix
import pandas as pd


pat = "dataset-12"
adata1 = sc.read_visium(path=pat, count_file='151676_filtered_feature_bc_matrix.h5')


/home/g_nemesis/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/g_nemesis/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [166]:
adata1

AnnData object with n_obs × n_vars = 3460 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [167]:
adata1
adata1.var_names_make_unique()

In [168]:
sc.pp.highly_variable_genes(adata1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata1, target_sum=1e4)
sc.pp.log1p(adata1)

In [169]:
adata1

AnnData object with n_obs × n_vars = 3460 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'spatial', 'hvg', 'log1p'
    obsm: 'spatial'

In [170]:
adata1.var

,gene_ids,feature_types,genome,highly_variable,highly_variable_rank,means,variances,variances_norm
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000
FAM138A,ENSG00000237613,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000
OR4F5,ENSG00000186092,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000
AL627309.1,ENSG00000238009,Gene Expression,GRCh38,False,NaN,0.002312,0.002307,0.974895
AL627309.3,ENSG00000239945,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
AC233755.2,ENSG00000277856,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000
AC233755.1,ENSG00000275063,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000
AC240274.1,ENSG00000271254,Gene Expression,GRCh38,False,NaN,0.001445,0.001443,0.975228
AC213203.1,ENSG00000277475,Gene Expression,GRCh38,False,NaN,0.000000,0.000000,0.000000


In [171]:
adata1 = adata1[:, adata1.var['highly_variable']]
adata1

View of AnnData object with n_obs × n_vars = 3460 × 3000
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'spatial', 'hvg', 'log1p'
    obsm: 'spatial'

In [172]:
# adata1.obs.index

In [173]:
# adata1.obsm["spatial"]

In [174]:
# import pickle
# data = list(adata1.var.feature_types.index)
# with open("features.pkl","wb") as file:
#     pickle.dump(data,file)

In [175]:
# adata1.obs.index

In [176]:
import pickle
barcodes = list(adata1.obs.index)
with open("barcodes.pkl","wb") as file:
    pickle.dump(barcodes,file)

In [177]:
import pickle
data = list(adata1.var.feature_types.index)
with open("features.pkl","wb") as file:
    pickle.dump(data,file)

In [178]:
import pandas as pd
import scipy.io as sio

pd.DataFrame(adata1.X.toarray())

sparse_matrix = adata1.X.tocoo()
output_file = 'matrix.mtx'
sio.mmwrite(output_file, sparse_matrix)

In [ ]:
import gzip
import shutil

# List of files you want to compress
files_to_compress = ['matrix.mtx']

# Loop through each file and compress it
for file_to_compress in files_to_compress:
    with open(file_to_compress, 'rb') as f_in:
        with gzip.open(file_to_compress + '.gz', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)



In [ ]:
from PIL import Image
# import json


# with open('scalefactors_json.json', 'r') as json_file:
#     scaling_factors = json.load(json_file)


input_image = Image.open('tissue_hires_image.png')  
input_image.save('tissue_hires_image.tif') 